In [1]:
import numpy as np
import pandas as pd
import torch
import pickle

import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', '..')))
from Functions_generation import generate_a_song_structure, sample_with_temp_topk, DecoderOnlyTransformer

/home/hlm/Documents/Stanford Course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("../../../Corpus/Encoding_RNN_LSTM/Char_level/encoding_map.pkl", "rb") as f:
    mapping = pickle.load(f)

int2char = {i: ch for ch, i in mapping.items()}
nb_char = len(int2char)
mapping["PAD"] = len(mapping)

In [3]:
matrix = pd.read_csv("../../../Markov/transition_matrix.csv")

states = np.array(matrix.iloc[6])
prob_transi = np.array(matrix.iloc[0:6])

In [4]:
vocab_size = len(mapping)

model = DecoderOnlyTransformer(vocab_size)   

In [5]:
ckpt = torch.load("../Models/MHA_model.pt", map_location="cpu")

new_state_dict = {}
for key, value in ckpt["model_state_dict"].items():
    new_key = key.replace("_orig_mod.", "")           
    new_state_dict[new_key] = value

ckpt["model_state_dict"] = new_state_dict

model.load_state_dict(ckpt["model_state_dict"])

<All keys matched successfully>

I will give the same structure for RNN and LSTM to compare

In [6]:
struct = generate_a_song_structure(prob_transi.astype(float),states)

#struct = ['<BEGINNING>', '<COUPLET>', '<REFRAIN>', '<COUPLET>', '<REFRAIN>', '<END>']
struct = ['α', 'γ', 'ε', 'γ', 'ε', 'θ']

encoded = torch.tensor([mapping[c] for c in struct[1]], dtype=torch.long).unsqueeze(0)

['<BEGINNING>', '<INTRO>', '<COUPLET>', '<REFRAIN>', '<COUPLET>', '<COUPLET>', '<OUTRO>', '<END>']


Forbidden to generate

In [7]:
forbidden_generation = [i for i in range(64,72)]
[int2char[i] for i in forbidden_generation]

['α', 'β', 'γ', 'ε', 'ζ', 'η', 'θ', '€']

In [8]:
decod_structure = {"β" : "<INTRO>",
                   "γ" : "<COUPLET>",
                   "ε" : "<REFRAIN>",
                   "ζ" : "<PONT>",
                   "η" : "<OUTRO>",
                   "θ" : "<END>"}

In [10]:
context_gen = "voila pourquoi, "
context_encode = [mapping[i] for i in context_gen]
context_size = len(context_encode)

for i in range(len(struct) - 2):

    print()
    print(decod_structure[struct[i+1]])

    if i == 0 : 
        context_encode.insert(0,mapping[struct[i+1]])
        context_encode.insert(1,mapping["\n"])
        encoded = torch.tensor(context_encode, dtype=torch.long).unsqueeze(0)

        print("".join([int2char[i] for i in context_encode[1:]]), end="")

        out = model(encoded)
        next_input = encoded[:, -1].unsqueeze(0)  # shape (1, 1)

    else : 
        next_input = torch.tensor(mapping[struct[i+1]],dtype=torch.long).view(1,1)

    last_input = 0
    first_input = True

    while last_input != 12:
        with torch.no_grad():
            out = model(next_input)   # out shape: (1, 1, vocab_size)

            out[0, -1][forbidden_generation] = float("-inf")

        # 4. On échantillonne un token
            next_token = sample_with_temp_topk(out[0, -1], temperature=0.6, top_k=10)
            last_input = next_token.item()

        # 5. Conditions d’impression
            if first_input and last_input == 12:  # Si début et le modèle sort directement <END>
                last_input = 0
                continue

            elif first_input and last_input == 0:  # Si début et sortie = \n
                first_input = False
                forbidden_generation.extend([0])
                print(int2char[next_token.item()], end="")

            elif next_token.item() != 12:
                first_input = False
                print(int2char[next_token.item()], end="")

                if 0 in forbidden_generation:
                    forbidden_generation.pop(-1)

        # 6. Préparer la prochaine entrée
            next_input = next_token.view(1, 1)

print("\n", decod_structure[struct[-1]])



<COUPLET>

voila pourquoi, pon j'e
eres pa ma fai j'ais t couris s las lai de de ceras e est's mmes pais me les lles le les dane lais le le de t mete plat pounden, l ce meretac'an le paran l'a maisur les
j'le d'e le
con s mes d t le pas c'es m'pe la t j'e lle e le lant me l
launte
les d's darese pan j'e mes c'es me dis esare le painte j'en je
j'ai t cont s de le ches cre d le j's c d'es prairsust s qus, me t couai less ditons tres pr de
tenntour, stitan les le pontra pa le pame es te moui, mpa d'e tre le de
landis s mommoroue di pre pe lis ce que j'a mis s dis ce pas
luire dous le es des es de ma lest e les pres couren pa d'aiese lererr s
cac'e pacontenon man me d'one pa des pons de c'e j'ais de s pen, fis s, con le cre d's s t l ment pail'ais daistou de de pon d't d'e ce
trouime pavon ce
ou'es c'an s d'c'as trailess lanne aienn s cos e t l'ais j's t de lu'ois me s deme l'e dais de lande m'ame su dr te dis ptre des lais, t l'at s j'a ten e ve, je ais pes d's teue e, les pes lainen t t